# SIXT33N Phase 3: PCA/Classification for Voice Commands

### EECS16B: Designing Information Devices and Systems II, Summer 2020

Written by Nathaniel Mailoa and Emily Naviasky (2016). Updated by Julian Chan (2018), Peter Schafhalter (2019). Vin Ramamurti and Zain Zaidi (Fall 2019)

nmailoa@berkeley.edu &emsp; enaviasky@berkeley.edu &emsp; julianchan0928@berkeley.edu &emsp; pschafhalter@berkeley.edu

## Table of Contents

* [Introduction / Lab Note](#intro)
* [Part 0: Preparing your Launchpad](#part0)
* [Part 1: Setting up your Circuit](#part1)
* [Part 2: Data Collection](#part2)
* [Part 3: Data Preprocessing](#part3)
* [Part 4: PCA via SVD](#part4)
* [Part 5: Clustering Data Points](#part5)
* [Part 6: Testing your Classifier](#part6)
* [Appendix: Formatting Vectors for Energia](#appendix)

<a id='intro'></a>
## Introduction

SIXT33N is an obedient little robot that will follow the directions that you tell it. There are four moves that SIXT33N can make: move straight, move straight slowly, turn right, and turn left. However, SIXT33N does not speak human languages, and some words, like "left" and "right", sound very similar (a strong single syllable), while other words are easy to distinguish. Your job in this phase is to find four command words that are easy for SIXT33N to tell apart (consider syllables and intonation).

For phase 2, you will develop the PCA classifier that allows SIXT33N to tell the difference between the four commands. You will examine several different words, and determine which ones will be easiest to classify by PCA.

**For more background on the connection between SVD and PCA, please read the [lab note](https://drive.google.com/file/d/13Chh7TGRFo9G-pzbiXdh79HAHj-g3BpY/view?usp=sharing).**

### Side Note: Datasets in Machine Learning Applications
It is common practice, especially in machine learning applications, to split a dataset into a training set and a smaller test set (some common ratios for train:test are 80:20 or 70:30) when trying to make data-driven predictions/decisions. In this lab, we will collect data and split our dataset into 70% training data and 30% test data. 

### Overview of Classification Procedure
Once you have some sample data collected, we will:
1. Split our data into 2 sets: train_data and test_data
2. Perform PCA and look at how well it separates the train_data 
3. Once you have a set of four words that you like, you will compute the means for each of those four words in the PCA basis. We will classify each word according to which mean it is closest in Euclidean distance to. 
4. To see how well our classifier does on data it has never seen before (this is called generalization in machine learning), we will project test_data onto the same PCA basis as train_data, and find the mean that is closest in Euclidean distance to each data point. 
5. When you (and your GSI) are satisfied with the classifier's accuracy, you will port the classifier into the Launchpad code in Energia.

<b>
The goals of this phase are as follows:
- Generate envelope and utilize threshold to get snippets
- PCA + Classifier (4 commands)
- Check accuracy
- PCA projection on Launchpad
</b>

The checkpoints (marked **<span style="color:green">green</span>** in the Notebook) are:
- Checkpoint 1: First pass through PCA with training data; GSI feedback
- Checkpoint 2: Classification target met in Python using test data

Note that all of this is considered a single graded lab.

When humans distinguish words, they listen for temporal and frequency differences to determine what is being said. However, SIXT33N does not have the memory or the processing power to distinguish words nearly as well as our human brains, so we will have to choose much simpler features for SIXT33N to look at (syllables, intonation, magnitude).

When you think of speech signals, you might notice that the shape of the speech wave is a very distinctive part of each word. Taking just the shape of the magnitude of a signal is called enveloping, exemplified in the image below. So, we want to do some filtering to retrieve the envelope of the audio signal. We train the PCA off of just this envelope and build a classifier to classify new data points.

<center>
<img width="400px" src="images/proj-envelope.png">
</center>

<b>Keeping in mind that the words that look most different have different shapes (or different amplitudes varied over time), brainstorm four or five words that you think will sort well. Consider syllables, intonation, and length of the word.</b>

**<span style="color:red">What words are you going to try? Why?</span>** 

<center>
<img width="400px" src="images/proj-waveform.png">
</center>

"Good" Audio data has a high signal to noise ratio. Recording words while far away from the microphone may cause your intended word to blend in with background noise. However "oversaturation" of the audio signal (speaking too loudly and/or too closely into the mic) will distort the signal (Why?). You can use the oscilloscope on the front-end output to test for over/under saturation.

Answer:

<a id='part0'></a>
## <span style="color:navy">Part 0: Preparing your Launchpad</span>

**Disconnect the 5V jumper wire that's powering the MSP through the 9V Battery and 9V -> 5V regulator**. As before, make sure that the MSP is not simultaneously being powered by both the computer (via the USB) and the 9v battery. Otherwise, you risk frying your MSP.

For the remainder of this lab, the MSP can be powered by only the computer, via the USB. 


<a id='part1'></a>
## <span style="color:navy">Part 1: Setting up your Circuit</span>

### Materials
- Microphone front-end circuit
- Launchpad + USB

### Tasks
1. You won't be using the motor circuits for this lab. You can leave that part of the circuit unpowered. 
2. Make sure the 9V is still powering the 9V->5V and 9V->3.3V voltage regulators
3. **Use the oscilloscope to probe the output of the microphone circuit.** Make sure the waveform averages to 1.65V (halfway between 0V and 3.3V) and the amplitude is large enough.
    - Make a noise at the microphone; you should see the signal change to reflect the sound you just made. If you are close enough or loud enough, you should be able to get the peak-to-peak amplitude of your signal all the way up to 3.3V.
    - Hint: set the oscilloscope's x-axis to 10ms per division and y-axis to 1 volt per division.

### Front End Verification

1. Hook up your front end circuit. **Make sure to disconnect the 5V jumper from the Launchpad** or you might fry it since this will power the Launchpad from the power supply as well as from the USB cord.
2. Without powering your circuit form the power supply, connect your circuit to the Launchpad:
    - P6.0 to the microphone front end circuit output.
    - 3.3V pin to the 3.3V power rail of the breadboard (in particular to supply the buffer op-amp).
    - 5V pin to the 5V row where your MicBoard $V_{DD}$ is located.
    - GND pin to the ground rail of the breadboard.
    - You can keep your Launchpad plugged via USB as long as **YOUR JUMPER IS DISCONNECTED**.
3. Use the bench power supply to provide 5V to the circuit.
    - **<span color='red'>DO NOT FORGET TO SET THE CURRENT LIMIT.</span>**

### Micboard Retuning/Verification Tasks

We want to make sure our micboard is tuned to get as clean a signal as possible. Let's retune our micboard!

1. Upload `tune_mic.ino` to your Launchpad and hit `RST`. Open the Serial Plotter. 
2. Press the button at the bottom left of the Launchpad (`P2.1`) to calibrate the readings. **Make sure the mic board gain is maxed out while this is happening.** Check to see that the values look reasonable.
3. **Play a pure tone in the midrange** (1000 - 5000 Hz) on your phone at a reasonable volume. Make sure to point the speaker directly at the mic and hold it a few inches away from the mic. Use a [tone generator](https://www.szynalski.com/tone-generator/) to do this, NOT a youtube video! The frequency in youtube videos is often inaccurate.

4. Turn the potentiometer (with a screwdriver or jumper pins) until you see a sine wave with a **peak-to-peak of over 2.5V** on the serial plotter. The plotter should have a legend at the top telling you which lines correspond to the mic board output, the peak-to-peak, and the 2.5V threshold. The peak-to-peak value is also given beside its legend title. If you tap on the mic, it should rail, but **it should not be railing while you play the tone.** Railing is nonlinear behavior and can negatively affect the performance of the rest of your circuit.
    - If you want to change the amount of time over which the peak-to-peak value is calculated, change the DELAY variable in the Launchpad code.
    - If the peak-to-peak reading stops being accurate, move the probe wire (the wire connecting the mic board output to `P6.0`) to the 3.45V rail and press `RST`. After the red and green LEDs at the bottom of the board turn off, you can move the probe wire back to the mic board output. 
    - If there's a non-physical spike on the serial plotter preventing you from seeing your voltages at the correct scale, try closing and reopening it. You may have to repeat the procedure outlined in the bullet point above. 
    

<span style="color:#ba190f">**Congratulations! You have successfully tuned your mic board!**</span>

<a id='part2'></a>
## <span style="color:navy">Part 2: Data Collection</span>

### Materials
- Microphone front-end circuit
- Launchpad + USB

### Tasks

#### Data Collection 

<span style="color:#ba190f">**Do this if you have a working micboard. Move onto Part 3: Data Preprocessing if you don't.**</span>

Now we will record 30 audio samples for each of your 4-5 chosen words.

**For each chosen word, do the following:**
1. Upload the sketch **`collect-data-envelope.ino`** to your Launchpad.
    - This sketch records 2 seconds of audio sampled every 0.35ms at a time and sends it to your computer.
    - When the red LED is on, the Launchpad is recording audio.
2. Run **`python collect-data-envelope.py YOUR_WORD.csv`**.
    - This program will capture audio data collected by the Launchpad and write it to `YOUR_WORD.csv`.
    - To debug, you can probe the output of the front end circuit with the `tune_mic.ino` program on your Launchpad and watch your Serial Plotter while you collect data. After you collect a few test words check `YOUR_WORD.csv` and make sure that it looks like a sound wave as opposed to being full of 0s. *It might help to plot the data in Excel.*
3. **When the red light goes on, say the word you want to record.**
    - **Pronounce the word consistently and always speak around the same time relative to when the red light turns on.** This will help you collect data that is less "noisy" which will result in better classification.
    - The red LED on the launch pad is like a recording room. When the red light goes on, the Launchpad is recording. Say the word you want to record before the red LED turns off.
4. Once you've recorded 30 audio samples of the word, stop the python program (e.g. by pressing Ctrl + C in the command prompt).


### Before moving on, please note that:

You may realize in the next section that one or two of your words are not sorting quite as well as you would like. Don't be afraid to come back to this section and try collecting different words based on what you have learned makes a word sortable. 

<a id='part3'></a>
## <span style="color:navy">Part 3: Data Preprocessing</span>

Before we can use the recorded data for PCA, we must first process the data. It is not necessary for you to understand the enveloping function well enough to implement it (since we have already done it for you), but just in case you are curious the enveloping function is described in the following pseudocode:

<code><b>Enveloping function</b>
Divide the whole signal to a block of 16 samples
For each chunk:
    Find the mean of the chunk
    Subtract each sample by the mean
    Find the sum of the absolute value of each sample
</code>

What you really need to know, however, is what the enveloped signal looks like for each word. Spend a little time looking at the data you just collected in the python plots below.

**If you didn't collect your own data, look in the `example_pca_data` directory for data that has been collected for your use.**

### 3.1 Load Data from CSV

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import csv
import utils
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [ ]:
# Load data from csv
# YOUR CODE HERE: fill in the path to the CSV file
word1_raw = utils.read_csv("")
word2_raw = utils.read_csv("")
word3_raw = utils.read_csv("")
word4_raw = utils.read_csv("")

# Split the data into training and test set
train_test_split_ratio = 0.7
word1_raw_train, word1_raw_test = utils.train_test_split(word1_raw, train_test_split_ratio)
word2_raw_train, word2_raw_test = utils.train_test_split(word2_raw, train_test_split_ratio)
word3_raw_train, word3_raw_test = utils.train_test_split(word3_raw, train_test_split_ratio)
word4_raw_train, word4_raw_test = utils.train_test_split(word4_raw, train_test_split_ratio)

# Take the same number of readings for all words to be fair
num_samples_train = min(np.shape(word1_raw_train)[0], np.shape(word2_raw_train)[0], np.shape(word3_raw_train)[0], np.shape(word4_raw_train)[0])
word1_raw_train = word1_raw_train[:num_samples_train,:]
word2_raw_train = word2_raw_train[:num_samples_train,:]
word3_raw_train = word3_raw_train[:num_samples_train,:]
word4_raw_train = word4_raw_train[:num_samples_train,:]

Plot your data and get a feel for how it looks enveloped.

**<span style="color:red">Important: It's okay if the data isn't aligned. The code in the next part will automatically align the data.</span>** 

In [ ]:
# Plot all training samples
plt.plot(word1_raw_train.T)
plt.show()
plt.plot(word2_raw_train.T)
plt.show()
plt.plot(word3_raw_train.T)
plt.show()
plt.plot(word4_raw_train.T)
plt.show()

### 3.2 Align Audio Recordings

As you can see above, the speech is only a small part of the 2 second window, and each sample starts at different times. PCA is not good at interpreting delay, so we need to somehow start in the same place each time and capture a smaller segment of the 2 second sample where the speech is present. To do this, we will use a thresholding algorithm.

First, we define a **`threshold`** relative to the maximum value of the data. We say that any signal that crosses the threshold is the start of a speech command. In order to not lose the first couple samples of the speech command, we say that the command starts **`pre_length`** samples before the threshold is crossed. We then take a window of the data that is **`length`** long, and try to capture the entire sound of the command in that window.

<b>Play around with the parameters `length`, `pre_length` and `threshold`</b> in the cells below to find appropriate values corresponding to your voice and chosen commands. You should see the results and how much of your command you captured in the plots generated below. When you are satisfied, note down the values of `length`, `pre_length` and `threshold` - <b>you will need to add them to the Launchpad sketch later.</b>

In [ ]:
def get_snippets(data, length, pre_length, threshold):
    """Attempts to align audio samples in data.
    
    Args:
        data (np.ndarray): Matrix where each row corresponds to a recording's audio samples.
        length (int): The length of each aligned audio snippet.
        pre_length (int): The number of samples to include before the threshold is first crossed.
        threshold (float): Used to find the start of the speech command. The speech command begins where the
            magnitude of the audio sample is greater than (threshold * max(samples)).
    
    Returns:
        Matrix of aligned recordings.
    """
    assert isinstance(data, np.ndarray) and len(data.shape) == 2, "'data' must be a 2D matrix"
    assert isinstance(length, int) and length > 0, "'length' of snippet must be an integer greater than 0"
    assert 0 <= threshold <= 1, "'threshold' must be between 0 and 1"
    snippets = []

    # Iterate over the rows in data
    for recording in data:
        # Find the threshold
        recording_threshold = threshold * np.max(recording)

        # Figure out when interesting snippet starts
        i = pre_length
        while recording[i] < recording_threshold:
            i += 1
            
        snippet_start = min(i - pre_length, len(recording) - length)
        snippet = recording[snippet_start:snippet_start + length]

        # Normalization
        snippet = snippet / np.sum(snippet)
        
        snippets.append(snippet)

    return np.vstack(snippets)

In [ ]:
length = 80 # Default: 80        # Adjust this
pre_length = 5 # Default: 5     # Adjust this
threshold = 0.5 # Default:  0.5    # Adjust this

word1_processed_train = get_snippets(word1_raw_train, length, pre_length, threshold)
plt.plot(word1_processed_train.T)
plt.show()
plt.figure()
word2_processed_train = get_snippets(word2_raw_train, length, pre_length, threshold)
plt.plot(word2_processed_train.T)
plt.show()
word3_processed_train = get_snippets(word3_raw_train, length, pre_length, threshold)
plt.plot(word3_processed_train.T)
plt.show()
plt.figure()
word4_processed_train = get_snippets(word4_raw_train, length, pre_length, threshold)
plt.plot(word4_processed_train.T)
plt.show()

You should now see a mostly organized set of samples for each word. Can you tell the which word is which just by the envelope? Can you tell them apart? If you can't tell the words apart, then PCA will have a difficult time as well.

<a id='part4'></a>
## <span style="color:navy">Part 4: PCA via SVD</span>

### 4.1 Generate and Preprocess PCA Matrix

Now that we have our data in a nice format, we can build the PCA input matrix from that data. The function <b>`np.vstack`</b> might be helpful here.

In [ ]:
# YOUR CODE HERE
processed_A = ...

Write code below to perform SVD on your matrix A (is there a function in one of our libraries that can help?), plot the sigma values, and project on to the principal components. First zero-mean your data as `demeaned_A`.

In [ ]:
# Zero-mean the matrix A
# YOUR CODE HERE
mean_vec = ...
demeaned_A = ...
print(processed_A.shape)
print(mean_vec.shape)

### 4.2 Principle Component Analysis

In [ ]:
# Take the SVD of matrix demeaned_A
# YOUR CODE HERE #
U, S, Vt = ...

In [ ]:
# Plot out the sigma values (Hint: Use plt.stem for a stem plot)
# YOUR CODE HERE #

Take a look at your sigma values. They should show you very clearly how many principal components you need.

**<span style="color:red">How many principal components do you need? Given that you are sorting 4 words, what is the the number you expect to need?</span>** 

There is no correct answer here. We can pick as many principal components onto which we project our data to get the "best" separation (most variance), but at some point, the cost-benefit isn't worth selecting an extra basis vector. For example, in our project, we are loading these basis vectors onto the [MSP430 Launchpad](http://www.ti.com/tool/MSP-EXP430F5529LP), and we can only store 2-3 principal components before we run into memory issues.

### 4.3 Choosing a Basis using Principle Components

Set the `new_basis` argument to be a basis of these principal components. (*Hint: Of the three outputs from the SVD function call, which one will contain the principal components onto which we want to project our data points?*)

When you plot `new_basis` you should see a number of line plots equal to the number of principal components you've chosen.

In [ ]:
# Plot the principal component(s)
# YOUR CODE HERE
new_basis = ...        # This should be the basis containing your principal components
plt.plot(new_basis)

Now project the data in the matrix A onto the new basis and plot it. Do you see clustering? Do you think you can separate the data easily? If not, you might need to try new words.

In [ ]:
def plot_3D(ax,data, view_from_top=False, m = 'o', si = 20):
    colors = ['blue', 'red', 'green', 'orange']
    for dat, color in zip(data, colors):
        Axes3D.scatter(ax, *dat.T, c=color, marker = m, s=si)
    if view_from_top:
        ax.view_init(elev=90.,azim=0)  # Move perspective to view from top

In [ ]:
# Project the data onto the new basis
# YOUR CODE HERE
proj = ...

'''Uncomment this block for 3 basis vectors 
fig=plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')
plot_3D(ax,[proj[:num_samples_train], proj[num_samples_train:2*num_samples_train], proj[2*num_samples_train:3*num_samples_train], proj[3*num_samples_train:4*num_samples_train]])
plt.show()
'''


fig=plt.figure(figsize=(10,7))
plt.scatter(proj[0:num_samples_train,0], proj[0:num_samples_train,1], c=['blue'], edgecolor='none')
plt.scatter(proj[num_samples_train:num_samples_train*2,0], proj[num_samples_train:num_samples_train*2,1], c=['red'], edgecolor='none')
plt.scatter(proj[num_samples_train*2:num_samples_train*3,0], proj[num_samples_train*2:num_samples_train*3,1], c=['green'], edgecolor='none')
plt.scatter(proj[num_samples_train*3:num_samples_train*4,0], proj[num_samples_train*3:num_samples_train*4,1], c=['orange'], edgecolor='none')
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()


Your data might look noisy, and might not classify perfectly. That is completely okay, we are just looking for good enough. Like many AI applications, this is noisy data that we are classifying so some error in classification is okay. The important part is if you think that you can see some clustering. 

Once you think you have decent clustering, you can move on to getting your code to automate classification and you will make up for some of the error there, too.

<a id='part5'></a>
## <span style="color:navy">Part 5: Clustering Data Points</span>

#### Implement `find_centroids` which finds the center of each cluster.

In [ ]:
def find_centroids(clustered_data):
    """Find the center of each cluster by taking the mean of all points in a cluster.
    It may be helpful to recall how you constructed the data matrix (e.g. which rows correspond to which word)
    
    Parameters:
        clustered_data: the data already projected onto the new basis
        
    Returns: 
        The centroids of the clusters
    """
    centroids = []
    # YOUR CODE HERE
    
    return centroids

In [ ]:
# Determine the centroids of each cluster
# YOUR CODE HERE
centroids = ...
print(centroids)

In [ ]:
centroid1 = centroids[0]
centroid2 = centroids[1]
centroid3 = centroids[2]
centroid4 = centroids[3]
centroid_list = np.vstack([centroid1, centroid2, centroid3, centroid4])

print('The first centroid is at: ' + str(centroid1))
print('The second centroid is at: ' + str(centroid2))
print('The third centroid is at: ' + str(centroid3))
print('The fourth centroid is at: ' + str(centroid4))

''' Uncomment this for 3 basis vectors: 
fig=plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')
plot_3D(ax,[proj[:num_samples_train], proj[num_samples_train:2*num_samples_train], proj[2*num_samples_train:3*num_samples_train], proj[3*num_samples_train:4*num_samples_train]], view_from_top=False)
plot_3D(ax,[np.array([centroids[0]]), np.array([centroids[1]]), np.array([centroids[2]]), np.array([centroids[3]])], False, '*', 300)
fig.show
'''




fig=plt.figure(figsize=(10,7))
plt.scatter(proj[0:num_samples_train,0], proj[0:num_samples_train,1], c=['blue'], edgecolor='none')
plt.scatter(proj[num_samples_train:num_samples_train*2,0], proj[num_samples_train:num_samples_train*2,1], c=['red'], edgecolor='none')
plt.scatter(proj[num_samples_train*2:num_samples_train*3,0], proj[num_samples_train*2:num_samples_train*3,1], c=['green'], edgecolor='none')
plt.scatter(proj[num_samples_train*3:num_samples_train*4,0], proj[num_samples_train*3:num_samples_train*4,1], c=['orange'], edgecolor='none')

plt.scatter(centroid_list[:,0], centroid_list[:,1], c=['blue', 'red', 'green', 'orange'], marker='*', s=300)
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Training Data")
plt.show()


<a id='part6'></a>
## <span style="color:navy">Part 6: Testing your Classifier</span>

Great! We now have the means (centroid) for each word. Now let's see how well our test data performs. Recall that we will classify each data point according to the centroid that it is closest in Euclidean distance to. 

Before we perform classification, we need to do the same preprocessing to the test data that we did to the training data (enveloping, demeaning, projecting onto the PCA basis). You have already written most of the code for this part. However, note the difference in variable names as we are now working with test data.

First let's look at what our raw test data looks like.

In [ ]:
# Take the same number of readings for all words to be fair
num_samples_test = min(np.shape(word1_raw_test)[0], np.shape(word2_raw_test)[0], np.shape(word3_raw_test)[0], np.shape(word4_raw_test)[0])
word1_raw_test = word1_raw_test[:num_samples_test,:]
word2_raw_test = word2_raw_test[:num_samples_test,:]
word3_raw_test = word3_raw_test[:num_samples_test,:]
word4_raw_test = word4_raw_test[:num_samples_test,:]

In [ ]:
# Plot all test samples
plt.plot(word1_raw_test.T)
plt.show()
plt.plot(word2_raw_test.T)
plt.show()
plt.plot(word3_raw_test.T)
plt.show()
plt.plot(word4_raw_test.T)
plt.show()

Perform enveloping and trimming of our test data.

In [ ]:
word1_processed_test = get_snippets(word1_raw_test, length, pre_length, threshold)
plt.plot(word1_processed_test.T)
plt.show()
plt.figure()
word2_processed_test = get_snippets(word2_raw_test, length, pre_length, threshold)
plt.plot(word2_processed_test.T)
plt.show()
word3_processed_test = get_snippets(word3_raw_test, length, pre_length, threshold)
plt.plot(word3_processed_test.T)
plt.show()
plt.figure()
word4_processed_test = get_snippets(word4_raw_test, length, pre_length, threshold)
plt.plot(word4_processed_test.T)
plt.show()

Construct the PCA matrix by stacking all the test data.

In [ ]:
# YOUR CODE HERE
processed_A_test = ...

**Now we will do something slightly different.**

Previously, you projected data using some projection matrix with $ P(x - \bar{x}) $, where $\bar{x}$ is the mean vector.

We can rewrite this operation as 

$$ P(x - \bar{x}) = Px - P \bar{x} = Px - \bar{x}_{\text{proj}} $$ 
$$ \bar{x}_{\text{proj}} = P \bar{x} $$

Why might we want to do this? We'll later perform these operations on our car. Our launchpads have limited memory, so we want to store as little as possible. Instead of storing a length $n$ vector $\bar{x}$, we can precompute $ \bar{x}_{\text{proj}} $ (length 2 or 3) and store that instead!

Compute $ \bar{x}_{\text{proj}} $ using the **same mean vector** as the one computed with the training data.

In [ ]:
# YOUR CODE HERE
projected_mean_vec = ...

Project the test data onto the **same PCA basis** as the one computed with the training data.

In [ ]:
# YOUR CODE HERE
projected_A_test = ...

Zero-mean the projected test data using the **`projected_mean_vec`**.

In [ ]:
# YOUR CODE HERE
proj = ...

Plot the projections to see how well your test data clusters in this new basis. This will give you an idea of how well your test data will classify.

In [ ]:
''' Uncomment this for 3 basis vectors
fig=plt.figure(figsize=(10,7))
ax = fig.add_subplot(111, projection='3d')
plot_3D(ax,[proj[:num_samples_test], proj[num_samples_test:2*num_samples_test], proj[2*num_samples_test:3*num_samples_test], proj[3*num_samples_test:4*num_samples_test]], view_from_top=False)
plot_3D(ax,[np.array([centroid_list[0]]), np.array([centroid_list[1]]), np.array([centroid_list[2]]), np.array([centroid_list[3]])], False, '*', 300)
fig.show
'''

fig=plt.figure(figsize=(10,7))
plt.scatter(proj[0:num_samples_test,0], proj[0:num_samples_test,1], c=['blue'], edgecolor='none')
plt.scatter(proj[num_samples_test:num_samples_test*2,0], proj[num_samples_test:num_samples_test*2,1], c=['red'], edgecolor='none')
plt.scatter(proj[num_samples_test*2:num_samples_test*3,0], proj[num_samples_test*2:num_samples_test*3,1], c=['green'], edgecolor='none')
plt.scatter(proj[num_samples_test*3:num_samples_test*4,0], proj[num_samples_test*3:num_samples_test*4,1], c=['orange'], edgecolor='none')
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))

plt.scatter(centroid_list[:,0], centroid_list[:,1], c=['blue', 'red', 'green', 'orange'], marker='*', s=300)
plt.legend(['word1', 'word2', 'word3', 'word4'],loc='center left', bbox_to_anchor=(1, 0.5))
plt.title("Test Data")
plt.show()

Now that we have some idea of how our test data looks in this new basis, let's see how our data actually performs. Implement the classify function that takes in a data point (AFTER enveloping is applied) and returns which word number it belongs to depending on which centroid the data point is closest in Euclidean distance to.

In [ ]:
def classify(data_point):
    """Classifies a new voice recording into a word.
    
    Args:
        data_point: new data point vector before demeaning and projection
    Returns:
        Word number (should be in {1, 2, 3, 4})
    Hint:
        Remember to use 'projected_mean_vec'!
    """
    # YOUR CODE HERE
    projected_data_point = ...
    demeaned = ...
    # TODO: classify the demeaned data point by comparing against the centroids

In [ ]:
# Try out the classification function
print(classify(processed_A_test[0,:])) # Modify to use other vectors

**Our goal is 80% accuracy for each word.** Write code to apply the `classify` function to each sample and compute the accuracy for each word.

In [ ]:
# Try to classify the whole A matrix
correct_counts = np.zeros(4)

for (row_num, data) in enumerate(processed_A_test):
    word_num = row_num // num_samples_test + 1
    if classify(data) == word_num:
        correct_counts[word_num - 1] += 1
        
for i in range(len(correct_counts)):
    print("Percent correct of word {} = {}%".format(i + 1, 100 * correct_counts[i] / num_samples_test))

<img width='30px' align='left' src="http://inst.eecs.berkeley.edu/~ee16b/sp16/lab_pics/check.png">

## <span style="color:green">CHECKOFF 2</span>

<span style="color:green">**Show your GSI that you've achieved 80% accuracy on your test data.** Your GSI will also check your PCA diagrams and code.</span>

<a id='appendix'></a>
## <span style="color:navy">Appendix: Formatting Vectors for Energia</span>

In next week's lab, copy/paste the following printed code into **`classify.ino`**

In [ ]:
print("Paste the code below into 'CODE BLOCK PCA1':")
print("")
print(utils.format_constant_energia("SNIPPET_SIZE", length))
print(utils.format_constant_energia("PRELENGTH", pre_length))
print(utils.format_constant_energia("THRESHOLD", threshold))

In [ ]:
print("Paste the code below into 'CODE BLOCK PCA2':")
print("")
print(utils.format_array_energia("pca_vec1", new_basis[:, 0]))
print(utils.format_array_energia("pca_vec2", new_basis[:, 1]))
# print(utils.format_array_energia("pca_vec3", new_basis[:, 2]))   # Uncomment this line if you have 3 PCA vectors
print(utils.format_array_energia("projected_mean_vec", projected_mean_vec))
print(utils.format_array_energia("centroid1", centroids[0]))
print(utils.format_array_energia("centroid2", centroids[1]))
print(utils.format_array_energia("centroid3", centroids[2]))
print(utils.format_array_energia("centroid4", centroids[3]))